In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import os
import random
import torch.optim as optim
import copy
import itertools
import pandas as pd
import json


# Seed and reproducibility


In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU
    
    # --- 2. Set CUDNN backend to be deterministic ---
    # This can slow down training, but is necessary for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    """
    Seeder function for DataLoader workers.
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# --- 3. Run the seeder ---
seed_everything(SEED)

# --- 4. Create a generator for DataLoaders ---
# We will pass this to all DataLoaders
g = torch.Generator()
g.manual_seed(SEED)

print(f"All seeds set to {SEED} and CUDNN is in deterministic mode.")

# Config and Parameter tuning


In [ ]:

CONFIG = {
    #  novedad -> We now have two models to save
    # --- MODIFIED: Changed to path templates ---
    "TEXT_VE_PATH_TPL": "best_text_ve_fold_{}.pth",
    "TRANSLATOR_PATH_TPL": "best_translator_mlp_fold_{}.pth",
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    
    # --- Single-Stage Training ---
    "EPOCHS": 150,
    "PATIENCE": 30,
    
    # --- NEW: K-Fold Configuration ---
    "N_FOLDS": 10,
    "RANDOM_STATE": 42,
    
    # --- Architecture/Loss Params ---
    "HIDDEN_FEATURES": 3072, 
    "NUM_BLOCKS": 2,
    "DROPOUT_RATE": 0.3832296399028748,     
    "LR": 1.7749224152998916e-05,             
    "WEIGHT_DECAY": 0.0008723639231133962,
    
    # --- Hybrid Loss Params ---
    "MARGIN": 0.3898481154806984,
    "TEMPERATURE": 0.010879304099273002,
    "HYBRID_ALPHA": 0,
    "MIXUP_ALPHA": 0.23197266859653176,
    
    #  novedad -> New VAE Params
    "LATENT_DIM": 2048,       # The "noisy" space between the two models
    "KLD_WEIGHT": 1.9543552300752537e-06,     # 
    
    # --- General Settings ---
    "BATCH_SIZE": 256,
    "ACCUMULATION_STEPS": 2
}

# Data Loading

In [ ]:
from sklearn.model_selection import KFold # <-- NEW: Import KFold

# --- 2. Data Loading ---
# We now load the *full* dataset here. 
# The splitting into train/val will be handled by KFold in the training loop.
try:
    full_train_data = np.load("/kaggle/input/train.npz", allow_pickle=True)
    all_text_embeddings = torch.from_numpy(full_train_data['captions/embeddings']).float()
    all_image_embeddings = torch.from_numpy(full_train_data['images/embeddings']).float()

    num_images = all_image_embeddings.shape[0]
    num_captions_per_image = all_text_embeddings.shape[0] // num_images

    # --- NEW: Define the KFold splitter ---
    # We split on image_indices to keep all captions for one image together
    image_indices = np.arange(num_images)
    kfold = KFold(n_splits=CONFIG['N_FOLDS'], shuffle=True, random_state=CONFIG['RANDOM_STATE'])

    print(f"Data loaded: {all_text_embeddings.shape[0]} text embeddings, {all_image_embeddings.shape[0]} image embeddings.")
    print(f"KFold splitter created with {CONFIG['N_FOLDS']} splits.")

except FileNotFoundError:
    print("Please make sure the 'train.npz' file is in the correct directory.")

# Model Architecture

In [ ]:
# --- 3. Model Architecture (V5 - VAE -> MLP) ---
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 3A. The GeGLU Activation Module ---
class GeGLU(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.proj = nn.Linear(in_features, out_features * 2)
        self.gelu = nn.GELU()
    def forward(self, x):
        proj_out = self.proj(x)
        a, b = proj_out.chunk(2, dim=-1)
        return self.gelu(a) * b

# --- 3B. The Residual Block with GeGLU ---
class ResidualBlockGeGLU(nn.Module):
    def __init__(self, features, dropout_rate):
        super().__init__()
        self.block = nn.Sequential(
            GeGLU(features, features), 
            nn.BatchNorm1d(features),
            nn.Dropout(dropout_rate)
        )
    def forward(self, x):
        return x + self.block(x)

#  novedad -> 3C. The *new* Text VAE
class TextVariationalEncoder(nn.Module):
    def __init__(self, in_features, hidden_features, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.GELU(),
            nn.LayerNorm(hidden_features)
        )
        self.fc_mu = nn.Linear(hidden_features, latent_dim)
        self.fc_log_var = nn.Linear(hidden_features, latent_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        return mu + epsilon * std

    def forward(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        z = self.reparameterize(mu, log_var)
        return z, mu, log_var

#  novedad -> 3D. The Translator (Your HybridMLP)
class TranslatorMLP(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, num_blocks, dropout_rate):
        super().__init__()
        
        # This model now takes the 'latent_dim' as its input
        backbone_layers = [
            nn.Linear(in_features, hidden_features, bias=False),
            nn.BatchNorm1d(hidden_features),
            nn.GELU(),
            nn.Dropout(0.5)
        ]
        for _ in range(num_blocks):
            backbone_layers.append(
                ResidualBlockGeGLU(hidden_features, dropout_rate) # Using GeGLU
            )
        self.backbone = nn.Sequential(*backbone_layers)
        
        self.translator_head = nn.Sequential(
            nn.Linear(hidden_features, hidden_features // 2),
            nn.GELU(),
            nn.Linear(hidden_features // 2, out_features) # out_features = 1536
        )

    def forward(self, x):
        shared_representation = self.backbone(x)
        output = self.translator_head(shared_representation)
        return F.normalize(output, p=2, dim=1)

# Loss Function

In [ ]:

class PureContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
    def forward(self, pred_norm, target_norm):
        sim_matrix = torch.matmul(pred_norm, target_norm.T) / self.temperature
        labels = torch.arange(pred_norm.size(0), device=pred_norm.device)
        return F.cross_entropy(sim_matrix, labels)

class TripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin
    def forward(self, text_embeds, img_embeds_norm):
        sim_matrix = text_embeds @ img_embeds_norm.T
        positive_scores = torch.diag(sim_matrix)
        mask = torch.eye(text_embeds.size(0), dtype=torch.bool, device=text_embeds.device)
        sim_matrix_masked = sim_matrix.masked_fill(mask, -float('inf'))
        hard_negative_scores = sim_matrix_masked.max(dim=1)[0]
        return F.relu(self.margin - positive_scores + hard_negative_scores).mean()

# --- 5B. FAST VALIDATION (for early stopping) ---
def validation_fn(text_ve, translator_model, val_loader, criterion_triplet, criterion_pure, alpha, kld_weight, device):
    text_ve.eval()
    translator_model.eval()
    val_loss = 0
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch, image_batch = text_batch.to(device), image_batch.to(device)
            
            #  novedad -> VAE -> MLP flow
            z, mu, log_var = text_ve(text_batch)
            pred_embeddings = translator_model(z)
            
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            
            # Calculate Hybrid Loss
            loss_triplet = criterion_triplet(pred_embeddings, target_embeddings)
            loss_pure = criterion_pure(pred_embeddings, target_embeddings)
            hybrid_loss = (alpha * loss_triplet) + ((1 - alpha) * loss_pure)
            
            # Calculate KLD Loss
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kld_loss = kld_loss / text_batch.size(0)
            
            loss = hybrid_loss + (kld_weight * kld_loss)
            val_loss += loss.item()
            
    return val_loss / len(val_loader)

# --- 5C. MRR CALCULATION (stable 1-vs-batch) ---
def calculate_mrr(text_ve, translator_model, val_loader, device):
    text_ve.eval()
    translator_model.eval()
    all_reciprocal_ranks = []
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch = text_batch.to(device)
            image_batch = image_batch.to(device)
            
            #  novedad -> VAE -> MLP flow
            z, _, _ = text_ve(text_batch) # We sample z
            pred_embeddings_norm = translator_model(z)
            
            target_embeddings_norm = F.normalize(image_batch, p=2, dim=1)
            sim_matrix = pred_embeddings_norm @ target_embeddings_norm.T
            
            correct_scores = torch.diag(sim_matrix).unsqueeze(1)
            ranks = (sim_matrix >= correct_scores).sum(dim=1).float()
            reciprocal_ranks = 1.0 / ranks
            all_reciprocal_ranks.append(reciprocal_ranks)
            
    mrr = torch.cat(all_reciprocal_ranks).mean().item()
    return mrr

# --- 5D. MIXUP ---
def mixup_data(x, y, alpha=0.2):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    batch_size = x.size(0)
    indices = torch.randperm(batch_size, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[indices, :]
    mixed_y = lam * y + (1 - lam) * y[indices, :]
    return mixed_x, mixed_y

# Main training

In [ ]:
all_fold_mrr = []

# =====================================================================================
#  NEW: K-FOLD OUTER LOOP
# =====================================================================================
for fold, (train_image_indices, val_image_indices) in enumerate(kfold.split(image_indices)):
    print(f"\n{'='*50}")
    print(f"--- STARTING FOLD {fold+1}/{CONFIG['N_FOLDS']} ---")
    print(f"{'='*50}")

    # --- 1. Create Datasets & DataLoaders for this fold ---
    
    # Get image embeddings for this fold
    train_image_emb = all_image_embeddings[train_image_indices]
    val_image_emb = all_image_embeddings[val_image_indices]

    # Create the expanded training image embeddings (one for each caption)
    train_image_emb_expanded = train_image_emb.repeat_interleave(num_captions_per_image, dim=0)

    # Get corresponding text caption embeddings for this fold
    train_caption_indices = [i for idx in train_image_indices for i in range(idx * num_captions_per_image, (idx + 1) * num_captions_per_image)]
    val_caption_indices = [i for idx in val_image_indices for i in range(idx * num_captions_per_image, (idx + 1) * num_captions_per_image)]

    train_text_emb = all_text_embeddings[train_caption_indices]
    val_text_emb = all_text_embeddings[val_caption_indices]
    
    # For validation, we only need one caption per image
    val_text_emb_one_per_image = val_text_emb[::num_captions_per_image]

    # Create TensorDatasets and DataLoaders
    train_dataset = TensorDataset(train_text_emb, train_image_emb_expanded)
    val_dataset = TensorDataset(val_text_emb_one_per_image, val_image_emb)

    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['BATCH_SIZE'], 
        shuffle=True,
        worker_init_fn=seed_worker,  # <-- Add this
        generator=g                 # <-- Add this
    )    
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'])

    print(f"Fold {fold+1}: Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

    # --- 2. Re-initialize Models, Losses, and Optimizer for this fold ---
    text_ve = TextVariationalEncoder(
        in_features=all_text_embeddings.shape[1],
        hidden_features=CONFIG['LATENT_DIM'],
        latent_dim=CONFIG['LATENT_DIM']
    ).to(CONFIG['DEVICE'])

    translator_model = TranslatorMLP(
        in_features=CONFIG['LATENT_DIM'],
        out_features=all_image_embeddings.shape[1],
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    criterion_triplet = TripletLoss(margin=CONFIG['MARGIN'])
    criterion_pure = PureContrastiveLoss(temperature=CONFIG['TEMPERATURE'])

    optimizer = optim.AdamW(
        itertools.chain(text_ve.parameters(), translator_model.parameters()),
        lr=CONFIG['LR'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )
    total_steps = len(train_loader) * CONFIG['EPOCHS']
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
    
    # --- 3. Training Loop for this fold ---
    best_finetune_mrr = -1.0
    patience_counter = 0

    for epoch in range(CONFIG['EPOCHS']):
        text_ve.train()
        translator_model.train()
        total_train_loss = 0
        
        # --- MODIFIED: Added fold info to tqdm description ---
        batch_iterator = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{CONFIG['EPOCHS']}")
        
        for i, (text_batch, image_batch) in enumerate(batch_iterator):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            image_batch = image_batch.to(CONFIG['DEVICE'])
            
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            mixed_text, mixed_targets = mixup_data(text_batch, target_embeddings, alpha=CONFIG['MIXUP_ALPHA'])

            z, mu, log_var = text_ve(mixed_text)
            pred_embeddings = translator_model(z)
            
            loss_triplet = criterion_triplet(pred_embeddings, mixed_targets)
            loss_pure = criterion_pure(pred_embeddings, mixed_targets)
            hybrid_loss = (CONFIG['HYBRID_ALPHA'] * loss_triplet) + ((1 - CONFIG['HYBRID_ALPHA']) * loss_pure)
            
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kld_loss = kld_loss / mixed_text.size(0)
            
            loss = hybrid_loss + (CONFIG['KLD_WEIGHT'] * kld_loss)
            loss = loss / CONFIG['ACCUMULATION_STEPS']
            loss.backward()

            if (i + 1) % CONFIG['ACCUMULATION_STEPS'] == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            total_train_loss += loss.item() * CONFIG['ACCUMULATION_STEPS']
            batch_iterator.set_postfix({"Train Loss": f"{loss.item() * CONFIG['ACCUMULATION_STEPS']:.4f}"})
            
        # --- Validation ---
        avg_train_loss = total_train_loss / len(train_loader)
        val_mrr = calculate_mrr(text_ve, translator_model, val_loader, CONFIG['DEVICE'])
        current_lr = scheduler.get_last_lr()[0]
        
        print(f"Fold {fold+1} Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f} | Val MRR: {val_mrr:.4f} | LR: {current_lr:e}")
        
        # train_loss_history.append(avg_train_loss) # Optional: you might want to average this across folds
        # finetune_mrr_history.append(val_mrr)
        
        if val_mrr > best_finetune_mrr:
            best_finetune_mrr = val_mrr
            
            # --- MODIFIED: Use the fold-specific path templates ---
            text_ve_path = CONFIG['TEXT_VE_PATH_TPL'].format(fold)
            translator_path = CONFIG['TRANSLATOR_PATH_TPL'].format(fold)
            
            torch.save(text_ve.state_dict(), text_ve_path)
            torch.save(translator_model.state_dict(), translator_path)
            print(f"  🏆 New best MRR for Fold {fold+1}! Saving models to {text_ve_path}...")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= CONFIG['PATIENCE']:
                print(f"Early stopping triggered for Fold {fold+1}.")
                break

    print(f"\n✅ Fold {fold+1} finished. Best validation MRR: {best_finetune_mrr:.4f}")
    all_fold_mrr.append(best_finetune_mrr)

print(f"\n{'='*50}")
print(f"✅ All {CONFIG['N_FOLDS']} folds trained! ")
print(f"Mean Validation MRR: {np.mean(all_fold_mrr):.4f} (Std: {np.std(all_fold_mrr):.4f})")
print(f"{'='*50}")

 # Submission

In [ ]:
print("--- Generating submission.csv") 

# --- 1. Load the test data ---
try:
    test_data = np.load("/kaggle/input/test.clean.npz", allow_pickle=True)
    test_text_emb = torch.from_numpy(test_data['captions/embeddings']).float()
    test_caption_ids = test_data['captions/ids']
    test_loader = DataLoader(test_text_emb, batch_size=CONFIG['BATCH_SIZE'])
    print(f"Test data and caption IDs loaded: {len(test_caption_ids)} samples.")
except FileNotFoundError:
    print("ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.")
    # Exit or raise error if you can't load test data
    
# --- 2. Loop through all folds to get predictions ---
all_predictions_list = []

for fold in range(CONFIG['N_FOLDS']):
    print(f"--- Generating predictions for Fold {fold+1}/{CONFIG['N_FOLDS']} ---")
    
    # --- Define model paths for this fold ---
    text_ve_path = CONFIG['TEXT_VE_PATH_TPL'].format(fold)
    translator_path = CONFIG['TRANSLATOR_PATH_TPL'].format(fold)

    # --- Load the best VAE and Translator models for this fold ---
    text_ve = TextVariationalEncoder(
        in_features=all_text_embeddings.shape[1],
        hidden_features=CONFIG['LATENT_DIM'],
        latent_dim=CONFIG['LATENT_DIM']
    ).to(CONFIG['DEVICE'])

    translator_model = TranslatorMLP(
        in_features=CONFIG['LATENT_DIM'],
        out_features=all_image_embeddings.shape[1], 
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    try:
        text_ve.load_state_dict(torch.load(text_ve_path))
        translator_model.load_state_dict(torch.load(translator_path))
    except FileNotFoundError:
        print(f"ERROR: Could not find model files for fold {fold}: {text_ve_path} or {translator_path}")
        print("Please ensure training is complete and files are saved correctly.")
        continue # Skip this fold if models aren't found
        
    text_ve.eval()
    translator_model.eval()

    # --- Create predictions in batches for this fold ---
    fold_predictions = []
    with torch.no_grad():
        for text_batch in tqdm(test_loader, desc=f"Predicting Fold {fold+1}"):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            
            # VAE -> MLP flow
            # We sample (z) just like in training/validation
            z, _, _ = text_ve(text_batch)
            pred_batch = translator_model(z)
            
            fold_predictions.append(pred_batch.cpu().numpy())
    
    all_predictions_list.append(np.concatenate(fold_predictions, axis=0))

if not all_predictions_list:
    print("\nNo predictions were generated. Cannot create submission file.")
else:
    # --- 3. Average the predictions from all folds ---
    print(f"\n--- Averaging predictions from {len(all_predictions_list)} folds ---")
    # Stack all prediction arrays (folds, samples, embedding_dim) and then mean across folds (axis=0)
    avg_predictions = np.mean(all_predictions_list, axis=0)
    
    # --- 4. Format for CSV submission ---
    embedding_json_list = [json.dumps(embedding.tolist()) for embedding in avg_predictions]

    submission_df = pd.DataFrame({
        'id': test_caption_ids,
        'embedding': embedding_json_list
    })

    # --- 5. Save the DataFrame to a CSV file ---
    submission_df.to_csv("submission.csv", index=False)

    print(f"\n✅ Ensembled submission file 'submission.csv' has been generated successfully!")
    print("Here's a preview of the first 5 rows:")
    print(submission_df.head())